In [22]:
from sklearn.cluster import DBSCAN
import numpy as np
from math import radians, cos, sin, asin, sqrt
import math

In [2]:
import json

In [25]:
x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626  # π
a = 6378245.0  # 长半轴
ee = 0.00669342162296594323  # 扁率
def gcj02towgs84(lng, lat):
    """
    GCJ02(火星坐标系)转GPS84
    :param lng:火星坐标系的经度
    :param lat:火星坐标系纬度
    :return:
    """
    if out_of_china(lng, lat):
        return lng, lat
    dlat = transformlat(lng - 105.0, lat - 35.0)
    dlng = transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng, lat * 2 - mglat]


def transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
        0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
            math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
            math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret


def transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
        0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
            math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
            math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret
def out_of_china(lng, lat):
    """
    判断是否在国内，不在国内不做偏移
    :param lng:
    :param lat:
    :return:
    """
    if lng < 72.004 or lng > 137.8347:
        return True
    if lat < 0.8293 or lat > 55.8271:
        return True
    return False
def get_distance_haversine(x,y):  # 经度1，纬度1，经度2，纬度2 （十进制度数）
    # 将十进制度数转化为弧度
    lon1, lat1 = x[0], x[1]
    lon2, lat2 = y[0], y[1]
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine公式
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # 地球平均半径，单位为公里
    return c * r * 1000

In [27]:
data = []
for i in range(82):
    f = open("data2/part-000%(num)02d"%{"num":i},"r")
    data.extend(f.readlines())
data2 = list(map(lambda x : x[1:-2],data))
data3 = []
origin = []
for item in data2:
    try:
        d = item.split(",")
        lng,lat = gcj02towgs84(float(d[0]),float(d[1]))
        data3.append([lng,lat])
        origin.append([float(d[0]),float(d[1])])
    except:
        print(item)
X = np.array(data3)

In [28]:
X.shape

(90998, 2)

In [30]:
X

array([[121.37730724,  31.25927349],
       [121.43884463,  31.27568759],
       [121.44388959,  31.26347973],
       ...,
       [121.45046696,  31.27581155],
       [121.40871648,  31.26319614],
       [121.44989885,  31.2547685 ]])

In [31]:
dbscan = DBSCAN(20,4,get_distance_haversine)
y = dbscan.fit_predict(X)
y.max()

3562

In [32]:
result = []
for i in range(len(data3)):
    result.append({
        "lnglat":origin[i],
        "style":(int(y[i]) + 1)%1000
    })

In [33]:
f = open("data2/points20-4.js","w")
f.write("var points = ")
f.write(json.dumps(result))
f.write(";")
f.close()